In [ ]:
!pip install -q langchain-community sentence-transformers google-generativeai langchain-google-genai faiss-cpu pypdf pdfminer.six

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 47.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.7/309.7 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 74.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 92.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 79.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56

In [ ]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
import torch
from google.colab import userdata

In [ ]:
GOOGLE_API_KEY=userdata.get("GOOGLE_API_KEY")
os.environ["GOOGLE_API_KEY"]=GOOGLE_API_KEY

In [ ]:
embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2",
                                    model_kwargs={'device': 'cuda' if torch.cuda.is_available() else 'cpu'},encode_kwargs={'normalize_embeddings': True},
                                )

/tmp/ipython-input-5-374063108.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2",
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to acce

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/402 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
llm = ChatGoogleGenerativeAI(model="gemini-pro",temperature=0.3,convert_system_message_to_human=True)

In [ ]:
# Create a special PDF loader customized for Bangla documents
class BanglaPDFLoader:


    def __init__(self, file_path):
        self.file_path = file_path  # Stores the PDF file location


    def load(self):
        try:
            # First try using PyPDFLoader
            loader = PyPDFLoader(self.file_path)
            pages = loader.load_and_split()  # Extract pages and split into sections

            # Print the extracted text from first page
            print("=== Extracted Text (PyPDF) ===")
            print(pages[0].page_content[:500] + "...")  # Show first 500 chars

            # Check if text contains Bangla characters
            if self._has_bangla_content(pages[0].page_content):
                return pages  # Return if Bangla found

        except Exception as e:
            # If PyPDF fails, try pdfminer
            print(f"PyPDF failed, trying pdfminer: {e}")

            from pdfminer.high_level import extract_text
            from langchain_core.documents import Document

            # Extract raw text using pdfminer
            text = extract_text(self.file_path)


            print("\n=== Extracted Text (pdfminer) ===")
            print(text[:500] + "...")


            return [Document(page_content=text)] if self._has_bangla_content(text) else []
        return []

    def _has_bangla_content(self, text):
        bangla_chars = set("অআইঈউঊঋএঐওঔকখগঘঙচছজঝঞটঠডঢণতথদধনপফবভমযরলশষসহড়ঢ়য়ৈােীুূৃৄেৈোৌ্ৎংঃ")
        return any(char in bangla_chars for char in text[:500])

In [ ]:
def clean_bangla_text(text):
  fixes = {
        "à¦": "া", "à§": "ে", "্র্": "্র", "্‌": "্",
        "": " ", "�": "", "\u200c": "", "\u200d": "",
        "  ": " ", "\t": " ", "\r\n": "\n",
        "ê": "ে", "ô": "ো", "û": "ূ", "î": "ী",
        "à": "া", "é": "ে", "ç": "চ"
    }
  for wrong,correct in fixes.items():
    text=text.replace(wrong,correct)
  return text.strip()

In [ ]:
def get_bangla_splitter():
    return RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        separators=["\n\n", "\n", "। ", "।", " ", ""],
        keep_separator=True,
        strip_whitespace=False
    )

In [ ]:
# 7. Bilingual Prompt Template
BILINGUAL_PROMPT = ChatPromptTemplate.from_messages([
    ("system", """
    You are a Bangla-English bilingual expert. Strictly follow:
    1. Respond in the QUESTION'S LANGUAGE
    2. For Bangla: Use চলিত ভাষা with numbers like ১০০
    3. Keep technical terms (e.g., "AI", "GDP") in original language
    4. If unsure: "প্রদত্ত ডকুমেন্টে তথ্য নেই" / "Info not in document"
    """),
    ("human", """
    DOCUMENT CONTEXT:
    {context}

    QUESTION:
    {question}

    ANSWER (concise and in the question's language):
    """)
])


In [ ]:
class BanglaRAGSystem:

  def __init__(self):
    self.embeddings=embeddings
    self.llm=ChatGoogleGenerativeAI(model="gemini-2.5-pro", temperature=0.3)
    self.vector_store=None

  def process_pdf(self,file_path):
    pages=BanglaPDFLoader(file_path).load()
    cleaned_pages = [clean_bangla_text(page.page_content) for page in pages if page.page_content.strip()]

    chunks=get_bangla_splitter().split_documents(pages)
    self.vector_store=FAISS.from_documents(chunks,self.embeddings)


    self.retriever = self.vector_store.as_retriever(
            search_type="mmr",  # Finds diverse relevant chunks
            search_kwargs={"k": 5, "lambda_mult": 0.6}  # Gets 5 best matches
        )

            # Create chain
    self.chain =(
            {"context": self.retriever, "question": RunnablePassthrough()}
            | BILINGUAL_PROMPT
            | self.llm
        )

  def query(self, question):
     return self.chain.invoke(question)

In [ ]:
if __name__ == "__main__":
    # Initialize
    rag = BanglaRAGSystem()

    # Upload PDF in Colab
    from google.colab import files
    print("Upload your Bangla PDF file:")
    uploaded = files.upload()
    pdf_file = next(iter(uploaded))

    # Process PDF
    rag.process_pdf(pdf_file)

    # Test questions
    questions = [
        "অনুপমের ভাষায় সুপুরুষ কাকে বলা হয়েছে?",
        "কাকে অনুপমের ভাগ্য দেবতা বলে উল্লেখ করা হয়েছে?",
        "বিয়ের সময় কল্যাণীর প্রকৃত বয়স কত ছিল?"
    ]

    for q in questions:
        print(f"\nপ্রশ্ন: {q}")
        print(f"উত্তর: {rag.query(q).content}")



Upload your Bangla PDF file:


Saving HSC26-Bangla1st-Paper.pdf to HSC26-Bangla1st-Paper (1).pdf
=== Extracted Text (PyPDF) ===
অনলাইন ব্যাচ সম্পর্কিত যেককাকনা জিজ্ঞাাসা ,
অপরিরিতা
আল ািয রিষয়
িাাং া
১ম পত্র...

প্রশ্ন: অনুপমের ভাষায় সুপুরুষ কাকে বলা হয়েছে?
উত্তর: প্রদত্ত ডকুমেন্ট অনুযায়ী, 'অপরিচিতা' গল্পে শম্ভুনাথকে সুপুরুষ বলা হয়েছে।

প্রশ্ন: কাকে অনুপমের ভাগ্য দেবতা বলে উল্লেখ করা হয়েছে?
উত্তর: প্রদত্ত ডকুমেন্ট অনুযায়ী, অনুপমের আসল অভিভাবক তার মামা, যিনি তাদের সংসারের সবকিছুর নিয়ন্ত্রক। তাই অনুপমের মামাকেই তার ভাগ্য দেবতা হিসেবে উল্লেখ করা হয়েছে।

প্রশ্ন: বিয়ের সময় কল্যাণীর প্রকৃত বয়স কত ছিল?
উত্তর: প্রদত্ত ডকুমেন্টে তথ্য নেই।


In [ ]:
import re

def clean_bangla_text(text):
    # Common encoding fixes (PDF/OCR/copy-paste errors)
    encoding_fixes = {
        # Fix vowel encoding issues
        "à¦": "া", "à§": "ে", "à¥": "ী", "à¤": "ি",
        "à¦¾": "া", "à¦¿": "ি", "à¦¸": "স",
        # Fix consonant clusters
        "্র্": "্র", "্রে": "্রে", "্রো": "্রো",
        # Fix invisible characters
        "\u200b": "", "\u200c": "", "\u200d": "", "\ufeff": "",
        # Fix broken punctuation
        "।।": "।", "..": "।", "?.": "?",
        # Fix common OCR mistakes
        "ê": "ে", "ô": "ো", "û": "ূ", "î": "ী",
        "à": "া", "é": "ে", "ç": "চ", "ñ": "ঞ"
    }

    # Apply encoding fixes
    for wrong, correct in encoding_fixes.items():
        text = text.replace(wrong, correct)

    # Step 2: Normalize Unicode compositions (e.g., ক + ্ + ষ = ক্ষ)
    text = text.replace("\u09CD\u09B7", "\u09CD\u09B7")  # ক্ষ fix

    # Step 3: Remove duplicate/irregular whitespace
    text = re.sub(r"\s+", " ", text).strip()

    # Step 4: Fix common conjuncts (যুক্তাক্ষর)
    conjunct_fixes = {
        "ক্‌ষ": "ক্ষ", "স্‌থ": "স্থ", "জ্‌ঞ": "জ্ঞ",
        "ব্‌দ": "ব্দ", "শ্‌র": "শ্র"
    }
    for wrong, correct in conjunct_fixes.items():
        text = text.replace(wrong, correct)

    return text

In [ ]:
print("\nইন্টারেক্টিভ মোড শুরু হয়েছে। প্রস্থান করতে 'exit' লিখুন")
while True:
    user_q = input("\nআপনার প্রশ্ন: ")
    cleaned_q = clean_bangla_text(user_q)

    if cleaned_q.lower() in ['exit', 'quit', 'প্রস্থান']:
        break

    response = rag.query(cleaned_q)
    print(f"\nউত্তর:\n{response.content}\n")


ইন্টারেক্টিভ মোড শুরু হয়েছে। প্রস্থান করতে 'exit' লিখুন

আপনার প্রশ্ন: অনুপলেি িািা কী কলি জীরিকা রনিবাহ কিলতন?

উত্তর:
প্রদত্ত ডকুমেন্টে তথ্য নেই।


আপনার প্রশ্ন: েীপুি িািাি সলে ‘অপরিরিতা' গ্লেি দকান িরিলেি রে আলে?

উত্তর:
প্রদত্ত ডকুমেন্ট অনুযায়ী, ‘অপরিচিতা’ গল্পের ‘অনুপমের মামা’ চরিত্রটির সাথে দীপুর মামার মিল রয়েছে।


আপনার প্রশ্ন: অনুপলেি িয়স কত িেি?

উত্তর:
প্রদত্ত ডকুমেন্টে তথ্য নেই।


আপনার প্রশ্ন: অনুপলেি িািা কী কলি জীরিকা রনিবাহ কিলতন?

উত্তর:
প্রদত্ত ডকুমেন্টে তথ্য নেই।


আপনার প্রশ্ন: োোলক ভাগ্য দেিতাি প্রধান এলজন্ট ি াি কািণ, তাি-

উত্তর:
প্রদত্ত ডকুমেন্ট অনুযায়ী, মামা হলেন অনুপমের আসল অভিভাবক এবং তিনি তাদের পুরো সংসারটি নিজের নিয়ন্ত্রণে রাখেন। অনুপম ব্যক্তিত্বহীন, পরিবারতন্ত্রের কাছে অসহায় এবং সবকিছুর জন্য মামার উপর নির্ভরশীল। মামার হীন মানসিকতা ও যৌতুকের গহনা পরীক্ষা করার মতো অবমাননাকর কাজের জন্যই অনুপমের বিয়ে ভেঙে যায়, যা তার জীবনের গতিপথ নির্ধারণ করে দেয়।


আপনার প্রশ্ন: exit


In [ ]:
import google.generativeai as genai
from google.colab import userdata


GOOGLE_API_KEY = userdata.get("GOOGLE_API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)

for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-flash-latest
models/gemini-1.5-flash
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-2.5-pro-preview-03-25
models/gemini-2.5-flash-preview-05-20
models/gemini-2.5-flash
models/gemini-2.5-flash-lite-preview-06-17
models/gemini-2.5-pro-preview-05-06
models/gemini-2.5-pro-preview-06-05
models/gemini-2.5-pro
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-preview-image-generation
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-2.0-pro-exp
models/gemini-2.0-pro-exp-02-05
models/gemini-exp-1206
models/gemini-2.0-flash-thinking-exp-01-21
models/gemin